## Preprocessing

In [18]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

EIN                                      NAME APPLICATION_TYPE  \
0  10520599              BLUE KNIGHTS MOTORCYCLE CLUB              T10   
1  10531628    AMERICAN CHESAPEAKE CLUB CHARITABLE TR               T3   
2  10547893        ST CLOUD PROFESSIONAL FIREFIGHTERS               T5   
3  10553066            SOUTHSIDE ATHLETIC ASSOCIATION               T3   
4  10556103  GENETIC RESEARCH INSTITUTE OF THE DESERT               T3   

        AFFILIATION CLASSIFICATION      USE_CASE  ORGANIZATION  STATUS  \
0       Independent          C1000    ProductDev   Association       1   
1       Independent          C2000  Preservation  Co-operative       1   
2  CompanySponsored          C3000    ProductDev   Association       1   
3  CompanySponsored          C2000  Preservation         Trust       1   
4       Independent          C1000     Heathcare         Trust       1   

      INCOME_AMT SPECIAL_CONSIDERATIONS  ASK_AMT  IS_SUCCESSFUL  
0              0                      N     5000              1  
1         1-9999                      N   108590              1  
2              0                      N     5000              0  
3    10000-24999                      N     6692              1  
4  100000-499999                      N   142590              1

In [19]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

In [20]:
# Determine the number of unique values in each column.
unique_app = application_df.nunique()
unique_app

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [21]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
app_type_counts = application_df['APPLICATION_TYPE'].value_counts()
app_type_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [22]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
threshold = 500
application_types_to_replace = app_type_counts[app_type_counts < threshold].index

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [23]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [24]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts_one = classification_counts[classification_counts > 1]
classification_counts_one

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [25]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
threshold = 500
classifications_to_replace = classification_counts[classification_counts < threshold].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: count, dtype: int64

In [26]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, drop_first=True)

In [27]:
# Split our preprocessed data into our features and target arrays
target = 'IS_SUCCESSFUL'
X = application_df.drop(columns=[target]).values
y = application_df[target].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [28]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [42]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation='sigmoid', input_dim=X_train_scaled.shape[1]))
nn.add(tf.keras.layers.BatchNormalization())  # Add Batch Normalization
nn.add(tf.keras.layers.Dropout(0.2))  # Regularization

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=100, activation='sigmoid'))
nn.add(tf.keras.layers.BatchNormalization())  # Add Batch Normalization
nn.add(tf.keras.layers.Dropout(0.2))  # Regularization

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation='sigmoid'))
nn.add(tf.keras.layers.BatchNormalization())  # Add Batch Normalization
nn.add(tf.keras.layers.Dropout(0.2))  # Regularization

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=100, activation='sigmoid'))
nn.add(tf.keras.layers.BatchNormalization())  # Add Batch Normalization
nn.add(tf.keras.layers.Dropout(0.2))  # Regularization

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                     │ (None, 80)                  │           3,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_12               │ (None, 80)                  │             320 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ (None, 80)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 100)                 │           8,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_13               │ (None, 100)                 │             400 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 80)                  │           8,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_14               │ (None, 80)                  │             320 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_14 (Dropout)                 │ (None, 80)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 100)                 │           8,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_15               │ (None, 100)                 │             400 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_15 (Dropout)                 │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 28,861 (112.74 KB)

 Trainable params: 28,141 (109.93 KB)

 Non-trainable params: 720 (2.81 KB)

In [43]:
# Compile the model
nn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
           loss='binary_crossentropy',
           metrics=['accuracy'])

In [44]:
# Train the model
train_model = nn.fit(X_train_scaled, y_train, epochs=15, batch_size=50, validation_split=0.2)

Epoch 1/15
440/440 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.6525 - loss: 0.6798 - val_accuracy: 0.7329 - val_loss: 0.5618
Epoch 2/15
440/440 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7006 - loss: 0.6017 - val_accuracy: 0.7382 - val_loss: 0.5539
Epoch 3/15
440/440 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7115 - loss: 0.5836 - val_accuracy: 0.7385 - val_loss: 0.5564
Epoch 4/15
440/440 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7180 - loss: 0.5755 - val_accuracy: 0.7360 - val_loss: 0.5550
Epoch 5/15
440/440 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.7163 - loss: 0.5745 - val_accuracy: 0.7382 - val_loss: 0.5542
Epoch 6/15
440/440 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7222 - loss: 0.5699 - val_accuracy: 0.7382 - val_loss: 0.5492
Epoch 7/15
440/440 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7262 - loss: 0.5636 - val_accuracy: 0.7385 - val_loss: 0.5498
Epoch 8/15
440/440 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7206 - loss: 0.5703 - val_accuracy: 0.

In [45]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 2ms/step - accuracy: 0.7254 - loss: 0.5585
Loss: 0.5585099458694458, Accuracy: 0.7253644466400146


In [46]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity3.h5')


In [47]:
import os
print(os.path.abspath('AlphabetSoupCharity3.h5'))



/content/AlphabetSoupCharity3.h5


In [48]:
from google.colab import files

# Download the model file
files.download('/content/AlphabetSoupCharity3.h5')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>